# Prácticas ESAC: Flury 2022 Data

1. LAEs Data: Flury 2022
2. Ly$\alpha$ luminosity
3. Escape fraction of Ly$\alpha$ photons: $f_{esc,Ly\alpha}$
4. Intrinsic Ly$\alpha$ photons
5. Number of ionizing photons $N_{ion}$

   Appendix A: Saving data

$\textit{Juan A. Molina-Calzada}$

In [1]:
# Packages
from astropy.table import QTable, join, unique
from astropy.io import ascii
import astropy.units as u
import astropy.cosmology.units as cu
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt
import numpy as np
import smplotlib


## 1. LAEs Data: Flury 2022

Leemos los datos de la tabla B1 de Kerutt et al. (2022). Cada fila se corresponde con una LAE.

In [2]:
# Data LAEs
table5 = QTable.read('table5_Flury22.txt', format = 'ascii', delimiter = ' ')
table1 = QTable.read('table1_Flury22.txt', format = 'ascii', delimiter = ' ')
table1_5 = unique(join(table1, table5, keys = 'ID'), keys = 'ID') # fusionamos table5 y tabl1 (nos interesa tener z en table5)

### 1.1. Distances calculation

Antes de empezar a trabajar con la línea de $Ly\alpha$, nos es necesario conocer las distancias de las galaxias a estudiar. Como conocemos su redshift, asumiendo un modelo cosmológico de $H_0 = 70 \, \text{km s}^{-1} \, \text{Mpc}^{-1}, \, \Omega_m = 0.3, \, \Omega_\Lambda = 0.7$, podemos estimar la distancia.


In [3]:
# Comoslogia asumida
H0 = 70 
Om0 = 0.3
cosmo = FlatLambdaCDM(H0, Om0)

distance = np.empty_like(table1_5['z'])

# Pasamos de redshift z a distancia en Mpc
for i in range(len(table1_5['z'])):
    distance[i] = cosmo.comoving_distance(table1_5['z'][i]).to(u.Mpc).value

table1_5['distance'] = np.round(distance,3)*u.Mpc

### 1.2. Continuum luminosity $L_{1200}$

A continuación, nos falta por conocer la luminosidad en el continuo $L_{1200}$. En este caso, dados los datos de la tabla, sólo conocemos el flujo f_{1100}. No obstante, podemos hacer la siguiente aproximación

$$
\begin{equation}
    f_{1200} \simeq f_{1100} = \frac{L_{1200}}{4\pi d^2}
\end{equation}
$$

utilizando la distancia calculada a partir del redshift z anteriormente.

In [4]:
# Calculamos la luminosidad en el continua (L_1200)
L1200 = np.round(table1_5['f1100'] *4 * np.pi * table1_5['distance']**2,2).to(u.erg/u.s/u.AA)
table1_5['logL1200'] = np.round(np.log10(L1200.value),3)*u.dex(u.erg/u.s/u.AA)

Estimamos también sus errores como

$$
\begin{equation}
    \Delta L_{1200} = 4\pi d^2 \Delta f_{1200}
\end{equation}
$$

donde $\Delta f_{1200}$ tiene dos incertidumbres dados los datos de la Tabla 5 de Flury22, una incertidumbre inferior, e_f1100, y otra superior, E_f1100.

In [5]:
# Incertidumbre inferior
e_logL1200 = (4 * np.pi * table1_5['distance']**2 * table1_5['e_f1100']).to(u.erg/u.s/u.AA) / (10**table1_5['logL1200'].value)
table1_5['e_logL1200'] = np.round(e_logL1200.value,3)
table1_5['e_logL1200'] = np.round(table1_5['e_logL1200'] * (1/np.log(10)),3)*u.dex(u.erg/u.s/u.AA)

# Incertidumbre superior
E_logL1200 = (4 * np.pi * table1_5['distance']**2 * table1_5['E_f1100']).to(u.erg/u.s/u.AA) / (10**table1_5['logL1200'].value)
table1_5['E_logL1200'] = np.round(E_logL1200.value,3)
table1_5['E_logL1200'] = np.round(table1_5['E_logL1200'] * (1/np.log(10)),3)*u.dex(u.erg/u.s/u.AA)

table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s))
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013


Ya tenemos todos los datos disponibles para continuar con los cálculos análogos a los realizados con los datos de Kerutt22.

## 2. Ly$\alpha$ luminosity

Conocida la anchura equivalente y la luminosidad en el continuo, podemos estimar la luminosidad de la línea Lyman alpha com

$$
\begin{equation}
     L_{Ly_\alpha,obs} = L_{1200} \cdot EW_0
\end{equation}
$$

donde 

$$
\begin{equation}
    L_{Ly\alpha,i} = 10^{log_{10}({L_{Ly\alpha,i}})}
\end{equation}
$$

In [6]:
# Luminosidad Lyman alpha
table1_5['logLLya'] = np.round(np.log10(10**table1_5['logL1200'].value * table1_5['EWLya'].value),3) * u.dex(u.erg/u.s)
table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034


### 2.1. Error in Ly$\alpha$ luminosity

Estimamos los errores en la luminosidad de Lyman alpha como

$$
\begin{equation}
    \Delta L_{Ly\alpha, obs} \equiv \Delta L_{Ly\alpha} = \sqrt{ \left(\Delta L_{1200} \cdot EW_0 \right)^2 + \left( L_{1200} \cdot \Delta EW_0 \right)^2}.
\end{equation}
$$

donde

$$
\begin{equation}
    \Delta log(x) = \frac{\Delta x}{x}
\end{equation}
$$

In [7]:
# Error en la luminosidad Lyman alpha
table1_5['e_logLLya'] = np.round((np.sqrt((table1_5['e_logL1200'].value * 10**table1_5['logL1200'].value * table1_5['EWLya'].value)**2 + (10**table1_5['logL1200'].value * table1_5['e_EWLya'].value)**2))/(10**table1_5['logLLya'].value),3)
table1_5['e_logLLya'] = np.round(table1_5['e_logLLya'] * (1/np.log(10)),3)*u.dex(u.erg/u.s)

table1_5['E_logLLya'] = np.round((np.sqrt((table1_5['E_logL1200'].value * 10**table1_5['logL1200'].value * table1_5['EWLya'].value)**2 + (10**table1_5['logL1200'].value * table1_5['e_EWLya'].value)**2))/(10**table1_5['logLLya'].value),3)
table1_5['E_logLLya'] = np.round(table1_5['E_logLLya'] * (1/np.log(10)),3)*u.dex(u.erg/u.s)

table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073


## 3. Escape fraction of Ly$\alpha$ photons: $f_{esc,Ly\alpha}$

Calculamos la fracción de escape de fotones Lyman alpha en función de la anchura equivalente (EW) para corroboral la relacion obtenida por Sobral et al. (2019). Nótese que la relación de Sobral sólo se cumple para 0<EW<200A.

$$
\begin{equation}
    f_{esc,Ly\alpha} = 0.0048(\pm 0.0007)EW_0 \pm 0.05
\end{equation}
$$

In [8]:
# Fracción de escape (Sobral)
f_esc_Lya = []

for i in range(len(table1_5['EWLya'])):
    if 0 <= table1_5['EWLya'][i].value <= 200:
        f_esc_Lya.append(0.0048*table1_5['EWLya'][i].value)
    else:
        f_esc_Lya.append(1)

# Add new columns with f_esc_Lya
table1_5['f_esc_Lya'] = np.round(f_esc_Lya,3)
table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039


### 3.1. Errors in fraction of Ly$\alpha$ photons

Calculamos ahora los errores de la función de escape de Lyman alpha acorde a la relación de Sobral et al. (2019).

$$
\begin{equation}
    \Delta f_{esc,Ly\alpha} = 0.0048(\pm 0.0007)\Delta EW_0
\end{equation}
$$

In [9]:
# Calculation of f_esc_Lya for all LAEs
e_f_esc_Lya = []

for i in range(len(table1_5['EWLya'])):
    if 0 <= table1_5['EWLya'][i].value <= 200:
        e_f_esc_Lya.append(0.0048*table1_5['e_EWLya'][i].value)
    else:
        e_f_esc_Lya.append(0)

# Add new columns with e_f_esc_Lya
table1_5['e_f_esc_Lya'] = np.round(e_f_esc_Lya,3)
table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya,e_f_esc_Lya
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),,
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451,0.045
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199,0.021
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15,0.017
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339,0.033
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19,0.023
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24,0.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039,0.007


## 4. Intrinsic Ly$\alpha$ photons

A continuación, calculamos la luminosidad de la línea de Lyman alpha a partir de la definición de función de escape:

$$
\begin{equation}
    f_{esc,Ly\alpha} = \frac{L_{Ly\alpha,obs}}{L_{Ly\alpha,int}}
\end{equation}
$$

donde 

$$
\begin{equation}
    L_{Ly\alpha,i} = 10^{log_{10}({L_{Ly\alpha,i}})}
\end{equation}
$$

In [10]:
# Luminosidad intríseca (propia del objeto) de Lyman alpha
table1_5['logLLya_int'] = np.round(np.log10(10**table1_5['logLLya'].value/table1_5['f_esc_Lya']),3)*u.dex(u.erg/u.s)
table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya,e_f_esc_Lya,logLLya_int
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),,,dex(erg / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451,0.045,42.478
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199,0.021,43.268
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15,0.017,43.129
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339,0.033,43.074
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19,0.023,43.201
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24,0.021,42.915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039,0.007,43.443


### 4.1. Erros in intrinsic Ly$\alpha$ photons

El error viene dado por:

$$
\begin{equation}
    \Delta L_{Ly\alpha,int} = \sqrt{\left(\frac{\Delta L_{Ly\alpha,obs}}{f_{esc,Ly\alpha}}\right)^2 + \left(\frac{L_{Ly\alpha,obs}} {f_{esc,Ly\alpha}^2}\Delta f_{esc,Ly\alpha}\right)^2}
\end{equation}
$$

donde

$$
\begin{equation}
    \Delta L_{Ly\alpha,obs} = \Delta log_{10}(L_{Ly\alpha,obs}) \cdot L_{Ly\alpha,int} = \Delta log_{10}(L_{Ly\alpha,obs}) \cdot 10^{log_{10}({L_{Ly\alpha,obs}})}
\end{equation}
$$

In [11]:
# Error en la luminsoidad Lyman alpha intríseca (propia del objeto)
table1_5['e_logLLya_int'] = np.round(np.sqrt((table1_5['e_logLLya'].value*10**table1_5['logLLya'].value/table1_5['f_esc_Lya'])**2+(10**table1_5['logLLya'].value*table1_5['e_f_esc_Lya']/table1_5['f_esc_Lya']**2)**2)/10**table1_5['logLLya_int'].value,3)
table1_5['e_logLLya_int'] = np.round(table1_5['e_logLLya_int'] * (1/np.log(10)),3)*u.dex(u.erg/u.s)
                                 
table1_5['E_logLLya_int'] = np.round(np.sqrt((table1_5['E_logLLya'].value*10**table1_5['logLLya'].value/table1_5['f_esc_Lya'])**2+(10**table1_5['logLLya'].value*table1_5['e_f_esc_Lya']/table1_5['f_esc_Lya']**2)**2)/10**table1_5['logLLya_int'].value,3)
table1_5['E_logLLya_int'] = np.round(table1_5['E_logLLya_int'] * (1/np.log(10)),3)*u.dex(u.erg/u.s)

table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya,e_f_esc_Lya,logLLya_int,e_logLLya_int,E_logLLya_int
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),,,dex(erg / s),dex(erg / s),dex(erg / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451,0.045,42.478,0.047,0.047
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199,0.021,43.268,0.05,0.05
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15,0.017,43.129,0.054,0.054
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339,0.033,43.074,0.046,0.046
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19,0.023,43.201,0.058,0.058
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24,0.021,42.915,0.042,0.043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039,0.007,43.443,0.084,0.084


## 5. Number of ionizing photons $N_{ion}$

Con los parámetros calculados, es el momento de calcular el número de fotones ionizantes, es decir, el número de fotones con $\lambda < 912$ Å (límite de Lyman). Utilizamos la siguiente expresión,

$$
\begin{equation}
    L_{Ly\alpha,int} = 1.19 \cdot 10^{-11} N_{ion}
\end{equation}
$$

asumiendo las condiciones del caso B de Osterbrock y Telectrónica~10.000 K (la normal en regiones ionizadas). Nótese que estamos abusando de notación y $N_{ion}$ = $N_{ion, L\alpha,int}$. 

In [12]:
# Numero de fotones ionizantes
table1_5['logN_ion'] = np.round(np.log10(10**table1_5['logLLya_int'].value / (1.19*10**(-11))),3)*u.dex(1/u.s)
table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya,e_f_esc_Lya,logLLya_int,e_logLLya_int,E_logLLya_int,logN_ion
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),,,dex(erg / s),dex(erg / s),dex(erg / s),dex(1 / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451,0.045,42.478,0.047,0.047,53.402
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199,0.021,43.268,0.05,0.05,54.192
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15,0.017,43.129,0.054,0.054,54.053
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339,0.033,43.074,0.046,0.046,53.998
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19,0.023,43.201,0.058,0.058,54.125
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24,0.021,42.915,0.042,0.043,53.839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039,0.007,43.443,0.084,0.084,54.367


### 5.1 Errors in number of ionizing photons $N_{ion}$

Calculamos los errores en el número de fotones ionizantes como

$$
\begin{equation}
    \Delta N_{ion} = \frac{\Delta L_{Ly\alpha,int}}{1.19 \cdot 10^{-11}}
\end{equation}
$$


In [13]:
# Error en el número de fotones ionizantes
table1_5['e_logN_ion'] = np.round((table1_5['e_logLLya_int'].value*10**table1_5['logLLya_int'].value / (1.19*10**(-11)))/(10**table1_5['logN_ion'].value),3)
table1_5['e_logN_ion'] = np.round(table1_5['e_logN_ion'] * (1/np.log(10)),3)*u.dex(1/u.s)

table1_5['E_logN_ion'] = np.round((table1_5['E_logLLya_int'].value*10**table1_5['logLLya_int'].value / (1.19*10**(-11)))/(10**table1_5['logN_ion'].value),3)
table1_5['E_logN_ion'] = np.round(table1_5['E_logN_ion'] * (1/np.log(10)),3)*u.dex(1/u.s)

table1_5

ID,RAdeg,DEdeg,z,Visit,Date,Exp,Num,M1500,e_M1500,r50,e_r50,beta1200,e_beta1200,f1100,e_f1100,E_f1100,EWLya,e_EWLya,logSFR,e_logSFR,distance,logL1200,e_logL1200,E_logL1200,logLLya,e_logLLya,E_logLLya,f_esc_Lya,e_f_esc_Lya,logLLya_int,e_logLLya_int,E_logLLya_int,logN_ion,e_logN_ion,E_logN_ion
,deg,deg,,,,s,,mag,mag,kpc,kpc,,,1e-19 W / (nm m2),1e-19 W / (nm m2),1e-19 W / (nm m2),0.1 nm,0.1 nm,dex(Msun / yr),dex(Msun / yr),Mpc,dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / (Angstrom s)),dex(erg / s),dex(erg / s),dex(erg / s),,,dex(erg / s),dex(erg / s),dex(erg / s),dex(1 / s),dex(1 / s),dex(1 / s)
str14,float64,float64,float64,str6,str10,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
J003601+003307,9.002641,0.552006,0.3479,ldxe08,2019-09-25,3980.384,13,-18.527,0.095,0.445,0.148,-2.9,0.328,6.445,0.381,0.418,93.9,9.33,0.038,0.028,1368.012,40.159,0.026,0.028,42.132,0.045,0.045,0.451,0.045,42.478,0.047,0.047,53.402,0.02,0.02
J004743+015440,11.928487,1.911086,0.3535,ldxe42,2019-07-29,1495.904,9,-20.732,0.103,0.618,0.145,-2.38,0.298,38.611,2.507,2.787,41.526,4.427,1.363,0.031,1388.004,40.949,0.028,0.031,42.567,0.048,0.048,0.199,0.021,43.268,0.05,0.05,54.192,0.022,0.022
J011309+000223,18.286905,0.039839,0.3062,ldxez5,2019-08-03,1363.904,13,-20.268,0.118,0.627,0.133,-1.99,0.253,36.421,3.649,2.623,31.291,3.56,1.203,0.031,1217.134,40.81,0.043,0.031,42.305,0.053,0.051,0.15,0.017,43.129,0.054,0.054,54.053,0.023,0.023
J012217+052044,20.569425,5.345561,0.3656,ldxe05,2019-09-20,3780.32,15,-20.143,0.096,0.713,0.151,-1.609,0.274,23.193,1.595,1.427,70.616,6.793,0.915,0.027,1430.981,40.755,0.03,0.026,42.604,0.044,0.043,0.339,0.033,43.074,0.046,0.046,53.998,0.02,0.02
J012910+145935,22.2923,14.992967,0.28,ldxe53,2019-06-11,1568.032,5,-20.439,0.089,0.636,0.127,-1.672,0.205,50.705,3.6,2.979,39.593,4.84,1.354,0.026,1120.513,40.882,0.031,0.026,42.48,0.055,0.054,0.19,0.023,43.201,0.058,0.058,54.125,0.025,0.025
J072326+414608,110.856661,41.768828,0.2969,ldxe66,2019-09-05,3894.304,12,-19.907,0.08,0.437,0.129,-1.22,0.174,23.56,1.893,3.198,49.995,4.421,1.184,0.06,1182.999,40.596,0.035,0.059,42.295,0.041,0.046,0.24,0.021,42.915,0.042,0.043,53.839,0.018,0.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J154050+572442,235.209141,57.411647,0.2944,ldxe25,2019-07-12,1768.032,16,-21.279,0.045,0.946,0.135,-1.433,0.117,80.832,2.415,2.382,8.113,1.368,1.873,0.013,1173.792,41.125,0.013,0.013,42.034,0.073,0.073,0.039,0.007,43.443,0.084,0.084,54.367,0.036,0.036


## Appendix A: Saving data

Si se ejecuta el siguiente código, guardaremos y sobreescribiremo los datos. Nótese que lo guardamos en formato ecsv (formato profesional para astrofísica)

In [14]:
ascii.write(table1_5, 'table1_5_Flury22_v1.dat', format='ecsv', overwrite=True)  
